<a href="https://colab.research.google.com/github/josecfreittas/ml-na-pratica/blob/master/semana01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
print("semana01")

semana01
